In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn import tree
import pydotplus
from sklearn.tree import DecisionTreeClassifier
from sklearn import datasets
from IPython.display import Image
from sklearn.cluster import AgglomerativeClustering 
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
import matplotlib as plt
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

In [2]:
profile=pd.read_csv('profile.csv')
profile
profile.describe()


,Unnamed: 0,age,became_member_on,income
count,17000.000000,17000.000000,1.700000e+04,14825.000000
mean,8499.500000,62.531412,2.016703e+07,65404.991568
std,4907.621624,26.738580,1.167750e+04,21598.299410
min,0.000000,18.000000,2.013073e+07,30000.000000
25%,4249.750000,45.000000,2.016053e+07,49000.000000
50%,8499.500000,58.000000,2.017080e+07,64000.000000
75%,12749.250000,73.000000,2.017123e+07,80000.000000
max,16999.000000,118.000000,2.018073e+07,120000.000000


In [3]:
profile.dropna(inplace=True)
profile.reset_index(drop=True)
profile.isnull().sum()

Unnamed: 0          0
gender              0
age                 0
id                  0
became_member_on    0
income              0
dtype: int64

In [4]:
# Code: convert date string to date


profile['became_member_on'] =\
    pd.to_datetime(profile.became_member_on, format="%Y%m%d")
print('done')


# test : convert date string to date

print(profile.became_member_on)

done
1       2017-07-15
3       2017-05-09
5       2018-04-26
8       2018-02-09
12      2017-11-11
           ...    
16995   2018-06-04
16996   2018-07-13
16997   2017-01-26
16998   2016-03-07
16999   2017-07-22
Name: became_member_on, Length: 14825, dtype: datetime64[ns]


In [5]:
 profile.rename(columns={"id": "customer_id"}, inplace=True)


In [6]:
trans=pd.read_csv('transcript.csv')
trans.describe()

,Unnamed: 0,time
count,306534.000000,306534.000000
mean,153266.500000,366.382940
std,88488.888045,200.326314
min,0.000000,0.000000
25%,76633.250000,186.000000
50%,153266.500000,408.000000
75%,229899.750000,528.000000
max,306533.000000,714.000000


In [7]:
 trans.rename(columns={"person": "customer_id"}, inplace=True)

In [8]:
def convert_time_day(val):
    """convert from time to days"""
    return val / 24


trans['days'] = trans.time.apply(convert_time_day)

trans

,Unnamed: 0,customer_id,event,value,time,days
0,0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,0.00
1,1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0,0.00
2,2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0,0.00
3,3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0,0.00
4,4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0,0.00
...,...,...,...,...,...,...
306529,306529,b3a1272bc9904337b331bf348c3e8c17,transaction,{'amount': 1.5899999999999999},714,29.75
306530,306530,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,{'amount': 9.53},714,29.75
306531,306531,a00058cf10334a308c68e7631c529907,transaction,{'amount': 3.61},714,29.75
306532,306532,76ddbd6576844afe811f1a3c0fbb5bec,transaction,{'amount': 3.5300000000000002},714,29.75


In [9]:
import ast
def string_to_dict(val):
    return ast.literal_eval(val)

trans.value = trans.value.apply(string_to_dict)

In [10]:
keys = []
def get_keys(val):
    """return available keys"""
    if list(val.keys()) not in keys:
        keys.append(list(val.keys()))
    
    
trans.value.apply(get_keys)

keys

keys = [ item for elem in keys for item in elem]
print(keys)

['offer id', 'amount', 'offer_id', 'reward']


In [11]:
def split_key(val):
    if list(val.keys())[0].startswith(key[:2]):
        return list(val.values())[0]
    else:
        return 0

for key in keys[1:3]:
    trans[key] = trans.value.apply(split_key)

In [12]:
trans.tail()

,Unnamed: 0,customer_id,event,value,time,days,amount,offer_id
306529,306529,b3a1272bc9904337b331bf348c3e8c17,transaction,{'amount': 1.5899999999999999},714,29.75,1.59,0
306530,306530,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,{'amount': 9.53},714,29.75,9.53,0
306531,306531,a00058cf10334a308c68e7631c529907,transaction,{'amount': 3.61},714,29.75,3.61,0
306532,306532,76ddbd6576844afe811f1a3c0fbb5bec,transaction,{'amount': 3.5300000000000002},714,29.75,3.53,0
306533,306533,c02b10e8752c4d8e9b73f918558531f7,transaction,{'amount': 4.05},714,29.75,4.05,0


In [13]:
offer = trans[trans.event == 'transaction']
offer

,Unnamed: 0,customer_id,event,value,time,days,amount,offer_id
12654,12654,02c083884c7d45b39cc68e1314fec56c,transaction,{'amount': 0.8300000000000001},0,0.00,0.83,0
12657,12657,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,{'amount': 34.56},0,0.00,34.56,0
12659,12659,54890f68699049c2a04d415abc25e717,transaction,{'amount': 13.23},0,0.00,13.23,0
12670,12670,b2f1cd155b864803ad8334cdf13c4bd2,transaction,{'amount': 19.51},0,0.00,19.51,0
12671,12671,fe97aa22dd3e48c8b143116a8403dd52,transaction,{'amount': 18.97},0,0.00,18.97,0
...,...,...,...,...,...,...,...,...
306529,306529,b3a1272bc9904337b331bf348c3e8c17,transaction,{'amount': 1.5899999999999999},714,29.75,1.59,0
306530,306530,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,{'amount': 9.53},714,29.75,9.53,0
306531,306531,a00058cf10334a308c68e7631c529907,transaction,{'amount': 3.61},714,29.75,3.61,0
306532,306532,76ddbd6576844afe811f1a3c0fbb5bec,transaction,{'amount': 3.5300000000000002},714,29.75,3.53,0


In [14]:
transactions = pd.merge(offer, profile, on='customer_id', how='left')
transactions

,Unnamed: 0_x,customer_id,event,value,time,days,amount,offer_id,Unnamed: 0_y,gender,age,became_member_on,income
0,12654,02c083884c7d45b39cc68e1314fec56c,transaction,{'amount': 0.8300000000000001},0,0.00,0.83,0,41.0,F,20.0,2016-07-11,30000.0
1,12657,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,{'amount': 34.56},0,0.00,34.56,0,55.0,M,42.0,2016-01-17,96000.0
2,12659,54890f68699049c2a04d415abc25e717,transaction,{'amount': 13.23},0,0.00,13.23,0,63.0,M,36.0,2017-12-28,56000.0
3,12670,b2f1cd155b864803ad8334cdf13c4bd2,transaction,{'amount': 19.51},0,0.00,19.51,0,134.0,F,55.0,2017-10-16,94000.0
4,12671,fe97aa22dd3e48c8b143116a8403dd52,transaction,{'amount': 18.97},0,0.00,18.97,0,151.0,F,39.0,2017-12-17,67000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
138948,306529,b3a1272bc9904337b331bf348c3e8c17,transaction,{'amount': 1.5899999999999999},714,29.75,1.59,0,16959.0,M,66.0,2018-01-01,47000.0
138949,306530,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,{'amount': 9.53},714,29.75,9.53,0,16964.0,M,52.0,2018-04-08,62000.0
138950,306531,a00058cf10334a308c68e7631c529907,transaction,{'amount': 3.61},714,29.75,3.61,0,16979.0,F,63.0,2013-09-22,52000.0
138951,306532,76ddbd6576844afe811f1a3c0fbb5bec,transaction,{'amount': 3.5300000000000002},714,29.75,3.53,0,16987.0,M,57.0,2016-07-09,40000.0


In [15]:
gk = transactions.groupby('customer_id') 
Money=gk['amount'].sum()
Freq=gk['customer_id'].count()
rec=gk['days'].min()
list_of_tuples = list(zip(gk.customer_id,rec,Freq,Money ))
rfm = pd.DataFrame(list_of_tuples,
                  columns=['CustomerID', 'Recency','Frequency','Monetary'])
rfm

,CustomerID,Recency,Frequency,Monetary
0,"(0009655768c64bdeb2e877511632db8f, [0009655768...",9.50,8,127.60
1,"(00116118485d4dfda04fdbaba9a87b5c, [0011611848...",12.25,3,4.09
2,"(0011e0d4e6b944f998e987f904e8c1e5, [0011e0d4e6...",5.50,5,79.46
3,"(0020c2b971eb4e9188eac86d93036a77, [0020c2b971...",2.25,8,196.86
4,"(0020ccbbb6d84e358d3414a3ff76cffd, [0020ccbbb6...",1.75,12,154.05
...,...,...,...,...
16573,"(fff3ba4757bd42088c044ca26d73817a, [fff3ba4757...",0.25,11,580.98
16574,"(fff7576017104bcc8677a8d63322b5e1, [fff7576017...",1.50,6,29.94
16575,"(fff8957ea8b240a6b5e634b6ee8eafcf, [fff8957ea8...",0.75,5,12.15
16576,"(fffad4f4828548d1b5583907f2e9906b, [fffad4f482...",1.50,12,88.83
